In [1]:
from bs4 import BeautifulSoup, Comment
import re
import os
import pandas as pd

In [2]:
def read_html_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    return html_content

def find_url_in_comments(html_content):
    soup = BeautifulSoup(html_content, 'lxml')
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        match = re.search(r'https?://\S+', comment)
        if match:
            return match.group()
    return None

def parse_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup

In [3]:
path = r'D:\github\garage\02_linkedin\download\\'

data = []

for file in os.listdir(path):
    if file.endswith('.html'):
        print(file)
        file_path = path + file
        
        html_content = read_html_file(file_path)
        link = find_url_in_comments(html_content)[:-1]
        soup = parse_html(html_content)
        
        name = soup.find('h1', {'class': 'text-heading-xlarge inline t-24 v-align-middle break-words'}).text.strip()

        try:
            headline = soup.find('div', {'class': 'text-body-medium break-words'}).text.strip()
        except:
            headline = None
        try:
            place = soup.find('span', {'class': 'text-body-small inline t-black--light break-words'}).text.strip()
        except:
            place = None
        try:
            company_link = soup.find('a', {'data-field': 'experience_company_logo'})['href']
        except:
            company_link = None
        try:
            company_name = soup.find('span', {
                'class': 'GrpjSuTnJuVIZvqQGOlBpxLRnnyFpfCYJCw hoverable-link-text break-words text-body-small t-black'}).text.strip()
        except:
            company_name = None
        try:
            position = soup.find('div', {'class': 'display-flex flex-wrap align-items-center full-height'}).text.strip()
        except:
            position = None
        try:
            education = soup.find('span', {
                'class': 'pv-text-details__right-panel-item-text hoverable-link-text break-words text-body-small t-black'}).text.strip()
        except:
            education = None

        profile_data = {
            'link': link,
            'name': name,
            'headline': headline,
            'place': place,
            'company_link': company_link,
            'company_name': company_name,
            'position': position,
            'education': education,
        }

        data.append(profile_data)

In [4]:
df = pd.DataFrame(data)
df.head()

""


In [5]:
len(df)

0

In [6]:
df.to_csv('profiles.csv', mode='a', header=False, index=False)

In [7]:
def get_links_list(profile_links_path, profiles_path):
    new_links = pd.read_csv(profile_links_path, usecols=['link'])

    if os.path.isfile(profiles_path):
        old_links = pd.read_csv(profiles_path, usecols=['link', 'name'])
        old_links.dropna(inplace=True)
    else:
        old_links = []

    return new_links.loc[~new_links['link'].isin(old_links['link']), 'link']

profile_links_path = 'profile_links_clean.csv'
profiles_path = 'profiles.csv'
links = get_links_list(profile_links_path, profiles_path)

len(links)

92

In [8]:
sum(links.duplicated())

0

In [9]:
links.to_csv('remain_links.csv', index=False)